In [205]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np 
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error
import ipaddress
import glob
import os
import tensorflow as tf
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split as tts      
from sklearn.preprocessing import StandardScaler as SS
from sklearn.metrics import confusion_matrix as CM 
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score as AS
from sklearn.linear_model import LinearRegression as LR
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.tree import DecisionTreeClassifier as dtc
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn import tree 

In [222]:
#DATA PROCESSING#

#Include all .csv files in directory of this code #

all_files = glob.glob("*.csv")
print(all_files)
li = []
#print(path)
data = pd.concat((pd.read_csv(f,index_col='No.') for f in all_files),ignore_index=True)

#Init Website Target#
data['Website'] = 100

#Website Target Labels and IP Addresses#
#SINGLE UP DOWN #

# (1-4) Amazon = 1
amaz_src_dest = ["18.160.17.214","18.165.77.79"] #TCP
data.loc[data['Source'] == amaz_src_dest[0],'Website'] = 1
data.loc[data['Source'] == amaz_src_dest[1],'Website'] = 1
data.loc[data['Destination'] == amaz_src_dest[0],'Website'] = 1
data.loc[data['Destination'] == amaz_src_dest[1],'Website'] = 1

# (1-5) Bing = 2
bing_src_dest = ["204.79.197.200"] #TCP
data.loc[data['Source'] == bing_src_dest[0],'Website'] = 2
data.loc[data['Destination'] == bing_src_dest[0],'Website'] = 2

# (1-4) ChatGPT = 3 
chat_src_dest = ["104.18.37.228"] # TCP
data.loc[data['Source'] == chat_src_dest[0],'Website'] = 3
data.loc[data['Destination'] == chat_src_dest[0],'Website'] = 3

#CONTINUOUS UP DOWN #
# (1-4) Discord CHAT CONTINUOUS = 4
discCHAT_src_dest = ["66.22.231.191"] # UDP
data.loc[data['Source'] == discCHAT_src_dest[0],'Website'] = 4
data.loc[data['Destination'] == discCHAT_src_dest[0],'Website'] = 4

# (1-3) Fortnite = 5
fort_src_dest = ["20.36.181.22","3.129.219.222","3.144.65.185"] # UDP
data.loc[data['Source'] == fort_src_dest[0],'Website'] = 5
data.loc[data['Source'] == fort_src_dest[1],'Website'] = 5
data.loc[data['Source'] == fort_src_dest[2],'Website'] = 5
data.loc[data['Destination'] == fort_src_dest[0],'Website'] = 5
data.loc[data['Destination'] == fort_src_dest[1],'Website'] = 5
data.loc[data['Destination'] == fort_src_dest[2],'Website'] = 5

# (1-4) Minecraft = 6 #TCP
mine_src_dest = ["209.222.115.47","104.234.169.167","31.25.11.18","31.25.11.55"]
data.loc[data['Source'] == mine_src_dest[0],'Website'] = 6
data.loc[data['Source'] == mine_src_dest[1],'Website'] = 6
data.loc[data['Source'] == mine_src_dest[2],'Website'] = 6
data.loc[data['Source'] == mine_src_dest[3],'Website'] = 6
data.loc[data['Destination'] == mine_src_dest[0],'Website'] = 6
data.loc[data['Destination'] == mine_src_dest[1],'Website'] = 6
data.loc[data['Destination'] == mine_src_dest[2],'Website'] = 6
data.loc[data['Destination'] == mine_src_dest[3],'Website'] = 6

#SINGLE DOWN #
# (1-5) Wikipedia = 0
wiki_src_dest = ["208.80.154.224"] #TCP
data.loc[data['Source'] == wiki_src_dest[0],'Website'] = 0 
data.loc[data['Destination'] == wiki_src_dest[0],'Website'] = 0 

#ryrob (1-3) = 7
ryrob_src_dest = ["172.66.40.147"] #QUIC
data.loc[data['Source'] == ryrob_src_dest[0],'Website'] = 7
data.loc[data['Destination'] == ryrob_src_dest[0],'Website'] = 7

#ugm.org (1-5) = 8
ugm_src_dest = ["13.84.36.2"] #TCP TLSv1.2
data.loc[data['Source'] == ugm_src_dest[0],'Website'] = 8
data.loc[data['Destination'] == ugm_src_dest[0],'Website'] = 8

#CONTINUOUS DOWN #
#(1-3) HULU = 9
hulu_src_dest = ["23.48.104.108", "23.48.104.112","23.48.104.110"] #TCP TLSv1.2
data.loc[data['Source'] == hulu_src_dest[0],'Website'] = 9
data.loc[data['Source'] == hulu_src_dest[1],'Website'] = 9
data.loc[data['Source'] == hulu_src_dest[2],'Website'] = 9
data.loc[data['Destination'] == hulu_src_dest[0],'Website'] = 9
data.loc[data['Destination'] == hulu_src_dest[1],'Website'] = 9
data.loc[data['Destination'] == hulu_src_dest[2],'Website'] = 9

#
data.drop(columns = ['Info'], inplace = True)
#Drop random IP addresses and Protocol
data = data.drop(data[data['Website'] == 100].index)
# data = data.drop(data[data['Protocol'] == "RTCP"].index)
#Convert IP addresses to Integers
data['Source'] = data['Source'].apply(lambda x : int(ipaddress.ip_address(x)))
data['Destination'] = data['Destination'].apply(lambda x : int(ipaddress.ip_address(x)))

#Convert Protocols to ints
#QUIC = 1
#TCP/TLS = 2
#UDP = 3 
#RTCP = 4
data.loc[data['Protocol'] == "QUIC",'Protocol'] = 1
data.loc[data['Protocol'] == "TLSv1.2",'Protocol'] = 2
data.loc[data['Protocol'] == "TLSv1.3",'Protocol'] = 2
data.loc[data['Protocol'] == "TCP",'Protocol'] = 2
data.loc[data['Protocol'] == "UDP",'Protocol'] = 3
data.loc[data['Protocol'] == "RTCP",'Protocol'] = 4


data.drop(columns = ['Source','Destination'], inplace = True)
data

['fortnite1_continuousupdown.csv', 'Wikipedia2_singledown.csv', 'hulu3_continuous-download.csv', 'Wikipedia5_singledown.csv', 'chatgpt3_singleupdown.csv', 'ryrod2_singledown.csv', 'ryrod3_singledown.csv', 'amazon1_singleupdown.csv', 'bing3_singleupdown.csv', 'Wikipedia4_singledown.csv', 'Wikipedia3_singledown.csv', 'discord3_continuousupdown.csv', 'Wikipedia1_singledown.csv', 'amazon4_singleupdown.csv', 'ryrod1_singledown.csv', 'chatgpt2_singleupdown.csv', 'bing_singleupdown.csv', 'chatgpt_singleupdown.csv', 'bing2_singleupdown.csv', 'discord2_continuousupdown.csv', 'minecraft1_continuousupdown.csv', 'minecraft4_continuousupdown.csv', 'ugm5_singledown.csv', 'minecraft2_continuousupdown.csv', 'bing5_singleupdown.csv', 'discord1_continuousupdown.csv', 'discord4_continuousupdown.csv', 'ugm2_singledown.csv', 'ugm3_singledown.csv', 'fortnite3_continuousupdown.csv', 'hulu2_continuous-download.csv', 'ugm4_singledown.csv', 'amazon3_singleupdown.csv', 'bing4_singleupdown.csv', 'hulu1_continuous

,Time,Protocol,Length,Website
0,0.000000,3,97,5
1,0.010276,3,82,5
2,0.048486,3,83,5
3,0.050166,3,120,5
4,0.076227,3,82,5
...,...,...,...,...
28173,6.244447,3,108,5
28174,6.250256,3,82,5
28175,6.280475,3,201,5
28176,6.315326,3,83,5


In [232]:
X = data.iloc[:,0:3]
y = data.iloc[:,3]


X_train, X_test, y_train, y_test = tts(X,y,random_state=17, test_size = 0.33)
 
# scaler = SS()  
#  # Don't cheat - fit only on training data
# scaler.fit(X_train) 

# X_train = scaler.transform(X_train)  
#  # apply same transformation to test data
# X_test = scaler.transform(X_test)  
#X_test.reshape(-1,5)


y

0        5
1        5
2        5
3        5
4        5
        ..
28173    5
28174    5
28175    5
28176    5
28177    5
Name: Website, Length: 27971, dtype: int64

In [219]:
#KNN
for i in range(2,10):
    knn = KNN(n_neighbors = i)

    knn.fit(X_train,y_train)
    y_pred = knn.predict(x_test)
    acc = AS(y_pred,y_test)
    print("accuracy{i}:",acc)

/Users/coltonturner/opt/anaconda3/envs/tensorflow/lib/python3.8/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


ValueError: Found array with dim 3. KNeighborsClassifier expected <= 2.

In [233]:
tree = dtc()
tree.fit(X_train,y_train)
y_pred = tree.predict(X_test)
CM(y_test,y_pred)

array([[ 148,   18,   27,   10,    0,    0,   33,    0,   26,   12],
       [   5,  397,   15,   19,    0,    0,   16,    0,   10,   19],
       [  30,   28, 1912,   22,    0,    0,   75,    0,   31,   39],
       [  14,   26,   18,  372,    0,    0,   12,    0,    6,   26],
       [   0,    0,    0,    0,  698,   47,    0,    9,    0,    0],
       [   0,    0,    0,    0,   42,  647,    0,    9,    0,    0],
       [  42,   21,   70,   24,    0,    0,  991,    0,   45,   46],
       [   0,    0,    0,    0,   10,    5,    0, 1310,    0,    0],
       [  37,    7,   37,    7,    0,    0,   29,    0,  771,    8],
       [  11,   24,   25,   18,    0,    0,   23,    0,   10,  842]])

In [234]:
#['Wikipedia','Amazon','Bing','Chat','Discord','Fortnite','Minecraft','ryrob','ugm','hulu']
print(classification_report(y_test,y_pred,target_names= ['Wikipedia','Amazon','Bing','Chat','Discord','Fortnite','Minecraft','ryrob','ugm','hulu']))
features = pd.DataFrame(tree.feature_importances_,index = X.columns)
features

              precision    recall  f1-score   support

   Wikipedia       0.52      0.54      0.53       274
      Amazon       0.76      0.83      0.79       481
        Bing       0.91      0.89      0.90      2137
        Chat       0.79      0.78      0.79       474
     Discord       0.93      0.93      0.93       754
    Fortnite       0.93      0.93      0.93       698
   Minecraft       0.84      0.80      0.82      1239
       ryrob       0.99      0.99      0.99      1325
         ugm       0.86      0.86      0.86       896
        hulu       0.85      0.88      0.87       953

    accuracy                           0.88      9231
   macro avg       0.84      0.84      0.84      9231
weighted avg       0.88      0.88      0.88      9231



,0
Time,0.481492
Protocol,0.236851
Length,0.281657


In [225]:
#TensorFlow Setup

features = data.drop(columns='Website').to_numpy()
labels = data['Website'].to_numpy()
features = np.expand_dims(features,-1)
labels = np.expand_dims(labels,-1)
#x_tensor = tf.convert_to_tensor(features, dtype=tf.int64) 
#y_tensor = tf.convert_to_tensor(labels, dtype=tf.int64) 
x_train, x_test,y_train,y_test = tts(features,labels,test_size=0.33,shuffle=True)
x_tensor = tf.convert_to_tensor(x_train, dtype=tf.int64) 
y_tensor = tf.convert_to_tensor(y_train, dtype=tf.int64) 
x_tensor
y_tensor



<tf.Tensor: shape=(18740, 1), dtype=int64, numpy=
array([[1],
       [4],
       [1],
       ...,
       [2],
       [8],
       [0]])>

In [200]:
model = tf.keras.Sequential([
    #keras.layers.Dense(len(x_tensor)),
    keras.layers.Dense(f),
    keras.layers.Dense(512, activation=tf.nn.relu),
    keras.layers.Dense(256, activation=tf.nn.relu),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(10,activation=tf.nn.softmax)
])
# model.add(tf.keras.layers.Dense(1024))
# model.add(tf.keras.layers.Dense(units=512, activation='relu'))
# model.add(tf.keras.layers.Dense(units=256, activation='relu'))
# model.add(tf.keras.layers.Dense(units=128, activation='relu'))
# model.add(tf.keras.layers.Dense(units=10, activation='softmax'))
# model = tf.keras.Sequential()
# model.add(tf.keras.layers.Dense(units=6, activation='relu', input_shape=[6,]))
# model.add(tf.keras.layers.Dense(units=20, activation='relu'))
# model.add(tf.keras.layers.Dense(units=25, activation='relu'))
# model.add(tf.keras.layers.Dense(units=20, activation='relu'))
# model.add(tf.keras.layers.Dense(units=1, activation='relu'))
    # keras.layers.Dense(x_tensor,kernel_regularizer=tf.keras.regularizers(0.1)),
    # keras.layers.Dense(512, activation=tf.nn.relu, kernel_regularizer=tf.kerars.regularizers(0.1)),
    # keras.layers.Dense(256, activation=tf.nn.relu, kernel_regularizer=tf.kerars.regularizers(0.1)),
    # keras.layers.Dense(128, activation=tf.nn.relu, kernel_regularizer=tf.kerars.regularizers(0.1)),
    # keras.layers.Dense(10,activation=tf.nn.softmax)

model.compile(optimizer=tf.compat.v1.train.AdamOptimizer(learning_rate=0.001),
            loss ='mse',
            metrics=['accuracy'])
# model.compile(optimizer = 'adam', loss = "mse",metrics=['accuracy'])
#model.summary()
history= model.fit(x_tensor,y_tensor,batch_size = 100,epochs = 400)

Epoch 1/400


InvalidArgumentError: Graph execution error:

Detected at node 'gradient_tape/mean_squared_error/BroadcastGradientArgs' defined at (most recent call last):
    File "/Users/coltonturner/opt/anaconda3/envs/tensorflow/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/Users/coltonturner/opt/anaconda3/envs/tensorflow/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/Users/coltonturner/opt/anaconda3/envs/tensorflow/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/Users/coltonturner/opt/anaconda3/envs/tensorflow/lib/python3.8/site-packages/traitlets/config/application.py", line 992, in launch_instance
      app.start()
    File "/Users/coltonturner/opt/anaconda3/envs/tensorflow/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 701, in start
      self.io_loop.start()
    File "/Users/coltonturner/opt/anaconda3/env

In [226]:
#Model Fitting
inputs = tf.keras.layers.Input(shape=(x_tensor.shape[1],x_tensor.shape[2]))
rnn_out = tf.keras.layers.SimpleRNN(32)(inputs)
outputs = tf.keras.layers.Dense(1)(rnn_out)

model = tf.keras.Model(inputs=inputs,outputs=[outputs])
model.compile(optimizer = 'adam', loss = "cross",metrics=['accuracy'])
model.summary()
history= model.fit(x_tensor,y_tensor,batch_size = 2,epochs = 4)

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 3, 1)]            0         
                                                                 
 simple_rnn_4 (SimpleRNN)    (None, 32)                1088      
                                                                 
 dense_101 (Dense)           (None, 1)                 33        
                                                                 
Total params: 1,121
Trainable params: 1,121
Non-trainable params: 0
_________________________________________________________________
Epoch 1/4
9370/9370 [==============================] - 13s 1ms/step - loss: 7.2564 - accuracy: 0.0535
Epoch 2/4
9370/9370 [==============================] - 13s 1ms/step - loss: 7.1583 - accuracy: 0.0535
Epoch 3/4
9370/9370 [==============================] - 12s 1ms/step - loss: 7.0488 - accuracy: 0.0535
Epoch 4/4
9370/